# Amazon Bedrock AgentCore Policy - 시작하기 데모

## 개요

Amazon Bedrock AgentCore Policy 실습 데모에 오신 것을 환영합니다! 이 노트북은 AI agent-tool 상호작용을 위한 정책 기반 결정론적 제어를 설정하고 테스트하는 전체 워크플로우를 안내합니다.

### AgentCore Policy란 무엇인가요?

Amazon Bedrock AgentCore Policy는 개발자가 agent 작업 주변에 보호 경계("안전 상자")를 생성하여 AI agent와 tool 간의 상호작용에 대한 보안 제어를 정의하고 시행할 수 있게 합니다. AI agent는 복잡한 문제를 해결하기 위해 동적으로 적응할 수 있지만, 이러한 유연성은 보안 문제를 야기합니다:

- **데이터 유출**: Agent가 실수로 개인 정보를 노출할 수 있음
- **비즈니스 규칙 위반**: Agent가 비즈니스 규칙을 잘못 해석하거나 우회할 수 있음
- **권한 초과**: Agent가 의도된 범위를 벗어나 행동할 수 있음

Policy는 AgentCore Gateway를 통해 인바운드 agent 트래픽을 가로채고 tool 액세스를 허용하기 전에 정의된 정책에 대해 각 요청을 평가합니다.

### 주요 이점

✅ **선언적 보안**: 코드가 아닌 Cedar 언어를 사용하여 정책 정의  
✅ **런타임 시행**: 정책이 실시간으로 평가됨  
✅ **세밀한 제어**: 대략적인 제한에서 상세한 액세스 제어/권한 부여까지
✅ **관심사 분리**: 보안 로직이 agent 코드 외부에 존재  
✅ **엔터프라이즈 규모**: 프로덕션에서 자율 agent를 안전하게 배포  

---

## 데모 아키텍처

```
┌─────────────┐
│   AI Agent  │
└──────┬──────┘
       │
       │ Tool Call Request
       ▼
┌─────────────────────┐
│  AgentCore Gateway  │
│  + OAuth Auth       │
└──────┬──────────────┘
       │
       │ Policy Check
       ▼
┌─────────────────────┐
│   Policy Engine     │
│   (Cedar Policies)  │
└──────┬──────────────┘
       │
       │ ALLOW / DENY
       ▼
┌─────────────────────--┐
│   Gateway Targets     │
│                       │    
└─────────────────────--┘
```

---

## 학습 내용

이 데모에서는 보험 인수를 수행하는 데 도움이 되는 tool을 가진 agent를 설정합니다:

1. **인프라 설정**: 보험 신청 생성, 리스크 model 호출 및 보험 청구 승인을 위한 Lambda target이 있는 Gateway 생성
2. **Policy Engine 생성**: gateway를 위한 policy engine 초기화
3. **정책 정의**: 액세스를 제어하기 위한 Cedar 정책 작성
4. **시행 테스트**: 실제 agent 요청으로 정책이 작동하는지 확인
5. **결과 이해**: ALLOW 및 DENY 시나리오 해석

---

## 사전 요구 사항

시작하기 전에 다음을 확인하세요:

- ✅ 적절한 자격 증명으로 구성된 AWS CLI
- ✅ boto3가 설치된 Python 3.10+
- ✅ `bedrock_agentcore_starter_toolkit` 패키지 설치
- ✅ AWS Lambda에 대한 액세스 (target 함수용)

---

## 데모 시나리오: 보험 인수 처리

정책 제어가 있는 **보험 인수 처리 시스템**을 구현합니다:

- **Tool**: 
       - `ApplicationTool` - 지역 및 coverage_amount에 대한 보험 신청 생성
       - `RiskModelTool` - API 분류 및 데이터 거버넌스 승인이 제공될 때 거버넌스 제어가 있는 외부 리스크 스코어링 model 호출
       - `ApprovalTool` - 고액 또는 고위험 인수 결정 승인, 매개변수 claim_amount 및 risk_level


시작해봅시다! 🚀

---

# Step 0: 환경 설정

먼저 환경을 확인하고 필요한 라이브러리를 가져옵니다.

In [ ]:
# Install requirements
%pip install -r requirements.txt

In [ ]:
# Import required libraries
import sys
from pathlib import Path  # 파일 경로 처리용
import boto3  # AWS SDK
import json
import logging

In [ ]:
# Add the scripts directory to Python path
scripts_dir = Path.cwd() / "scripts"  # 현재 디렉토리의 scripts 폴더 경로
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))  # Python 모듈 검색 경로에 추가

# Verify region
session = boto3.Session()
region = session.region_name or "us-east-1"  # 기본값으로 us-east-1 사용

# Verify AWS credentials
try:
    sts = session.client("sts")  # AWS Security Token Service 클라이언트
    identity = sts.get_caller_identity()  # 현재 AWS 자격 증명 정보 조회
    print("✅ AWS Credentials Verified")
    print(f"   Account: {identity['Account']}")
    print(f"   User/Role: {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS Credentials Error: {e}")
    print("   Please configure AWS CLI with: aws configure")

---

# Step 1: AgentCore Gateway를 위한 Target 함수 생성

gateway를 설정하기 전에 Agent의 tool target 역할을 할 함수가 필요합니다.

## Lambda Target이란 무엇인가요?

Lambda target은 AI agent가 gateway를 통해 호출할 수 있는 백엔드 함수입니다. 우리의 경우, agent가 보험 인수 작업을 수행하도록 지원하기 위해 ApplicationTool, RiskModelTool 및 ApprovalTool용 3개의 lambda 함수를 설정합니다.

### CLI를 통한 생성 (아래 셀 실행)
다음 스크립트를 실행하면 AWS 계정에 3개의 Lambda가 배포됩니다:

1. Application Tool: 간소화된 신청 생성 (데모 목적으로 모의 구현)
 신청자 지역 및 보장 금액으로 보험 신청 생성
 매개변수:
 - applicant_region: 고객의 지리적 지역
 - coverage_amount: 요청된 보험 보장 금액

2. Risk Model Tool: 간소화된 리스크 Model 액세스 (데모 목적으로 모의 구현)
 리스크 스코어링 model을 호출하고 평가 반환
 매개변수:
 - API_classification: API 분류 (public, internal, restricted)
 - data_governance_approval: 데이터 거버넌스가 model 사용을 승인했는지 여부

3. Approval Tool: - 보험 승인 프로세스 (데모 목적으로 모의 구현)
 인수 결정 및 청구 금액 승인
 매개변수:
 - claim_amount: 보험 청구/보장 금액
 - risk_level: 리스크 수준 평가 (low, medium, high, critical)

In [ ]:
%run scripts/lambda-target-setup/deploy_lambdas.py 

---

# Step 2: AgentCore Gateway 설정

이제 OAuth 인증을 사용하여 AgentCore Gateway를 생성하고 Lambda 함수를 target으로 연결합니다.

## 무엇이 생성되나요?

1. **OAuth Authorization Server**: 인증을 위한 Cognito 기반 OAuth
2. **AgentCore Gateway**: MCP 프로토콜 지원이 있는 메인 gateway
3. **Lambda Target**: 스키마 정의와 함께 연결된 Lambda 함수
4. **구성 파일**: 나중에 사용할 모든 연결 세부 정보 저장

## Gateway 구성

gateway는 다음과 같이 구성됩니다:
- **Protocol**: MCP (Model Context Protocol)
- **Authentication**: Cognito를 통한 OAuth 2.0
- **Target**: ApplicationTool, RiskModelTool, ApprovalTool Lambda 함수 - Target 스키마가 Gateway에 제공됨

In [ ]:
# Run the gateway setup script
print("🚀 Setting up AgentCore Gateway...\n")
print("This will:")
print("  1. Create OAuth authorization server (Cognito)")
print("  2. Create AgentCore Gateway")
print("  3. Attach Lambda as target")
print("  4. Save configuration to config.json")
print("\n" + "=" * 60)

# Run the gateway setup script
%run scripts/setup_gateway.py

### Gateway 구성 확인

방금 생성된 gateway 구성을 로드하고 확인해봅시다:

In [ ]:
# Load gateway configuration
gateway_config_file = "config.json"

with open(gateway_config_file, "r") as f:
    gateway_config = json.load(f)  # Gateway 설정 파일 로드

print("✅ Gateway Configuration Loaded\n")
print("=" * 60)
print(f"Gateway ID:  {gateway_config['gateway']['gateway_id']}")
print(f"Gateway ARN: {gateway_config['gateway']['gateway_arn']}")
print(f"Gateway URL: {gateway_config['gateway']['gateway_url']}")
print(f"Region:      {gateway_config['region']}")
print("\nOAuth Configuration:")
print(f"  Client ID:  {gateway_config['gateway']['client_info']['client_id']}")
print(f"  Token URL:  {gateway_config['gateway']['client_info']['token_endpoint']}")
print("=" * 60)

# Store for later use
GATEWAY_ARN = gateway_config["gateway"]["gateway_arn"]
GATEWAY_ID = gateway_config["gateway"]["gateway_id"]
GATEWAY_URL = gateway_config["gateway"]["gateway_url"]

### Gateway에 생성된 tool을 사용하는 agent 실행

In [ ]:
# Import the agent session
from scripts.agent_with_tools import AgentSession

# Use the agent within a context manager (this handles setup and cleanup automatically)
with AgentSession() as session:  # context manager로 자동 리소스 관리
    # The agent will list all available tools during setup

    # Now you can invoke the agent with different prompts
    response1 = session.invoke("What tools do you have access to?")

    response2 = session.invoke("Create an application for US region with $5M coverage")

    response3 = session.invoke(
        "Invoke the risk model with public API classification and data governance approval set to true"
    )

    response4 = session.invoke(
        "Approve underwriting for $75000 claim with medium risk level"
    )

# The session is automatically cleaned up when exiting the 'with' block
print("=" * 60)
print(
    f"🚀 The agent has access to the following tools configured on the Gateway: {response1}\n"
)
print(f"🚀 The agent can create applications without any limits: {response2}\n")
print(f"🚀 The agent can invoke the risk model: {response3}\n")
print(f"🚀 The agent is able to approve the insurance claims: {response4}\n")
print("=" * 60)

---

# Step 3: Policy Engine 및 정책 생성

이제 gateway의 tool에 대한 액세스를 제어하기 위해 Cedar 정책이 있는 Policy Engine을 생성합니다.

## Policy Engine이란 무엇인가요?

Policy Engine은 실시간으로 Cedar 정책에 대해 요청을 평가합니다. 두 가지 모드로 작동합니다:
- **LOG_ONLY**: 평가하지만 차단하지 않음 (테스트용)
- **ENFORCE**: 비준수 요청을 적극적으로 차단 (프로덕션용)

Gateway가 Policy Engine과 연결되면 기본 작업은 거부이며, 특정 정책이 액세스를 허용하지 않는 한 거부됩니다. 빈 policy engine은 이제 gateway의 어떤 tool도 액세스할 수 없습니다.

### Policy Engine 생성

먼저 Cedar 정책을 보관할 Policy Engine을 생성합니다:

In [ ]:
# Import PolicyClient from AgentCore Starter Toolkit
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

policy_client = PolicyClient(region_name=region)
policy_client.logger.setLevel(logging.INFO)  # 로그 레벨 설정

# Create a Policy Engine
print("🔧 Creating Policy Engine...")
engine = policy_client.create_or_get_policy_engine(
    name="InsurancePolicyEngine",
    description="Policy engine for insurance underwriting governance",
)
print(f"✓ Policy Engine: {engine['policyEngineId']}\n")

In [ ]:
# Save Policy Engine in the configuration file
with open("config.json", "r") as f:
    config = json.load(f)

# Add policy engine information (without removing existing data)
config["policy_engine_id"] = engine["policyEngineId"]
config["policy_engine_arn"] = engine["policyEngineArn"]

# Write back the updated config
with open("config.json", "w") as f:
    json.dump(config, f, indent=2)  # 들여쓰기 2칸으로 JSON 저장

print("✅ Policy engine information added to config.json")

In [ ]:
# Attach Policy Engine to the Gateway
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

gateway_client = GatewayClient(region_name=region)
gateway_client.logger.setLevel(logging.INFO)

gateway_client.update_gateway_policy_engine(
    gateway_identifier=config["gateway"]["gateway_id"],
    policy_engine_arn=engine["policyEngineArn"],
    mode="ENFORCE",  # 정책 위반 시 요청 차단
)
print("✓ Policy Engine attached to Gateway\n")

### 이제 Policy Engine이 Gateway에 연결되었으므로 기본적으로 tool이 차단되며, list tools API 호출 중에도 agent가 Gateway를 통해 액세스하려고 할 때도 차단됩니다

In [ ]:
# Import the agent session
from scripts.agent_with_tools import list_available_tools, fetch_access_token

client_info = config["gateway"]["client_info"]

CLIENT_ID = client_info["client_id"]
CLIENT_SECRET = client_info["client_secret"]
TOKEN_URL = client_info["token_endpoint"]
access_token = fetch_access_token(CLIENT_ID, CLIENT_SECRET, TOKEN_URL)  # OAuth 토큰 발급

print("=" * 60)
print("🚀 The agent has access to the following tools configured on the Gateway: \n")
print(list_available_tools(config["gateway"]["gateway_url"], access_token))

### Cedar Policy 생성

이제 보장 금액이 100만 미만인 경우 보험 신청을 생성할 수 있도록 허용하는 Cedar 정책을 생성합니다:

## 정책 규칙

```cedar
permit(
  principal,
  action == AgentCore::Action::"ApplicationToolTarget___create_application",
  resource == AgentCore::Gateway::"<gateway-arn>"
) when {
  context.input.coverage_amount <= 1000000
};
```

이것은 다음을 의미합니다:
- ✅ $1000000 이하의 보험 신청 생성: **허용됨**
- ❌ $1000000 초과의 보험 신청 생성: **거부됨**

In [ ]:
# Create Cedar policy
print("\n📝 Creating Cedar Policy...")
print(f"   Policy Engine ID: {engine['policyEngineArn']}")
GATEWAY_ARN = config["gateway"]["gateway_arn"]

# Define the Cedar policy statement
# 보장 금액이 100만 이하일 때만 신청 생성 허용
cedar_statement = (
    f"permit(principal, "
    f'action == AgentCore::Action::"ApplicationToolTarget___create_application", '
    f'resource == AgentCore::Gateway::"{GATEWAY_ARN}") '
    f"when {{ context.input.coverage_amount <= 1000000 }};"
)

policy = policy_client.create_or_get_policy(
    policy_engine_id=engine["policyEngineId"],
    name="create_application_policy",
    description="Allow application creation under $1M",
    definition={"cedar": {"statement": cedar_statement}},
)
print(f"✓ Policy: {policy['policyId']}\n")

# Save to config
config["policy_id"] = policy["policyId"]
with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

---

# Step 4: AI Agent로 정책 시행 테스트

이제 흥미로운 부분입니다 - 실제 AI agent로 정책을 테스트해봅시다!
또한 create_application tool에 대해 연결한 정책으로 인해 이제 이 tool이 gateway에 의해 나열되고 agent가 호출할 수 있음을 알 수 있습니다

## 테스트 시나리오

두 가지 시나리오를 테스트합니다:

### 테스트 1: 허용 시나리오 ✅
- **요청**: $750,000의 보장 금액으로 신청 생성
- **예상**: 정책이 허용하고, Lambda가 실행되며, 신청이 생성됨
- **이유**: $750K <= $1M (정책 제한 내)

In [ ]:
# Use the agent within a context manager (this handles setup and cleanup automatically)
with AgentSession() as session:
    # The agent will list all available tools during setup

    # Now you can invoke the agent with different prompts
    response1 = session.invoke("What tools do you have access to?")

    response2 = session.invoke(
        "Create an application for US region with $750,000 coverage"
    )

### 테스트 2: 거부 시나리오 ❌
- **요청**: $1.5M의 보장 금액으로 신청 생성
- **예상**: 정책이 차단하고, Lambda가 실행되지 않음
- **이유**: $1.5M > $1M (정책 제한 초과)

In [ ]:
with AgentSession() as session:
    # The agent will list all available tools during setup

    response2 = session.invoke(
        "Create an application for US region with $1.5M coverage"
    )

## 정리

<div style="background-color: #d1ecf1; border-left: 4px solid #0c5460; padding: 10px; margin: 10px 0; color: #000;">
    <strong style="color: #000;">ℹ️ 참고:</strong> 02-Natural-Language-Policy-Authoring/NL-Authoring-Policy.ipynb는 이 데모에서 설정한 Gateway를 재사용합니다. 정리 단계를 건너뛰고 두 번째 실습에서 NL2Cedar 기능을 테스트한 후 수행할 수 있습니다.
</div>


policy engine 및 정책의 리소스를 정리하려면 다음 순서로 수행됩니다:
1. update_gateway CLI를 사용하고 빈 policy engine을 전달하여 gateway의 policy engine 연결 삭제

2. policy engine의 모든 정책 삭제

3. policy engine 삭제

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

with open("config.json", "r") as f:
    config = json.load(f)

# Clean up Policy Engine first
print("🧹 Cleaning up Policy Engine...")
policy_client = PolicyClient(region_name=config["region"])
policy_client.cleanup_policy_engine(config["policy_engine_id"])  # 정책 및 엔진 삭제
print("✓ Policy Engine cleaned up\n")

# Then clean up Gateway
print("🧹 Cleaning up Gateway...")
gateway_client = GatewayClient(region_name=config["region"])
gateway_client.cleanup_gateway(
    config["gateway"]["gateway_id"], config["gateway"]["client_info"]
)  # Gateway 및 OAuth 리소스 삭제
print("✅ Cleanup complete!")